In [15]:
import pandas as pd
import  matplotlib
import matplotlib.pyplot as plt
from dqn_agent import Agent
import torch
import numpy as np
from main_new import getState, money_calculate, cost_calculate

STATE_SIZE = 20

data_dir = "StockData"
filename = "600967.SS.csv"
file_dir = data_dir + "/" + filename

try:
    df = pd.read_csv(file_dir, encoding='gbk')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_dir, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_dir, encoding='iso-8859-1')

In [19]:
df['date'] = pd.to_datetime(df['Date'])
df.set_index('date', inplace=True)

agent = Agent(state_size=STATE_SIZE, action_size=3)
agent.qnetwork_local.load_state_dict(torch.load('checkpoint_000997.pth', map_location=lambda storage, loc: storage))


stockData = list(df['Close'])
l = len(stockData)-1
window_size = 20
state = getState(stockData, 0, window_size + 1)
# total_profit = 0
agent.balance = []
action_list = []
pos_list = []
pos_old = 0
total_share = 0
cost = 0
money_initial = 10000
money = money_initial

In [21]:
for t in range(l):
    action = agent.act(state, eps=0)
    next_state = getState(stockData, t + 1, STATE_SIZE + 1)
    if action == 1:# 买入
        pos_new = min(pos_old + 0.2, 1)
        total_share += money * (pos_new - pos_old) / stockData[t]
        
    elif action == 2 :
        pos_new = max(pos_old - 0.2, 0)
        total_share += money * (pos_new - pos_old) / stockData[t]
    else:
        pos_new = pos_old
    money = money_calculate(money, total_share, stockData[t], pos_new)

    done = 1 if t == l - 1 else 0
    state = next_state
    pos_old = pos_new
    action_list.append(action)
    pos_list.append(pos_new)
    total_profit = (money - money_initial) / money_initial
    # value_list.append(stockData[t])
    if done:
        print("------------------------------")
        print("total_profit = " + str(total_profit))
        print("------------------------------")
        action_list.append(0)
        pos_list.append(pos_new)
        df['action'] = pd.DataFrame(action_list).values
        df['pos'] = pd.DataFrame(pos_list).values

        
        fig, ax1 = plt.subplots()
        

        df["Close"].plot(figsize=(8, 5), grid=True, label='1st')
        sell = (df['action'].values == 2)
        plt.scatter(df.index[sell], df["Close"].values[sell], c='r')
        buy = (df['action'].values == 1)
        plt.scatter(df.index[buy], df["Close"].values[buy], c='g')
        plt.legend(['value', 'sell', 'buy'])
        ax2 = ax1.twinx()
        plt.bar(df.index, df['pos'].values, label='2st')
        plt.ylim([0, 4])
        ax1.set_ylabel("Rate of Return")
        ax2.set_ylabel("Position")
        plt.show()




------------------------------
total_profit = 4.51559127991255
------------------------------


ValueError: Length of values (722) does not match length of index (361)